# Tests de primalidad

## Pseudoprimalidad

Para cada $a < n$, calculamos $gcd(a,n)$. Si no es $1$, o si $a^{n-1} \not\equiv_n 1$, devuelve falso. Sólo en caso contrario devuelve verdadero.

Los números de Carmichael son aquellos que, para cualquier coprimo, cumplen que $a^{n-1} \equiv_n 1$.

In [1]:
# Fija una semilla de aleatoriedad para asegurarse de
# la reproducibilidad de los tests
import random
random.seed(1)

In [2]:
from sympy import *

def isPseudoprime(a,n):
    
    """Comprueba si a es pseudoprimo módulo n. El argumento
    debe ser un entero."""
    
    return gcd(a,n) == 1 and pow(a,n-1,n) == 1

def pspTest(n, times=1):
    
    """Realiza un test de pseudoprimalidad sobre n usando
    como base un número aleatorio entre 2 y n. El argumento
    debe ser un entero mayor que 2. Opcionalmente se puede
    indicar el número de veces que puede realizarse."""
    
    # Repite el test hasta que encuentra un número contra el
    # que no es pseudoprimo. Sólo devuelve verdadero si no encuentra
    # ningún número en el test.
    for _ in xrange(times):
        a = random.randint(2,n-1)
        
        if not isPseudoprime(a,n):
            return False, a
    
    return True

In [3]:
pspTest(19,10)

True

## Símbolos de Jacobi y Legendre

#### Símbolo de Legendre
El símbolo de Legendre se define como:

$$
\left(\frac{a}{p}\right) = \left\{
\begin{array}{rl}
0 & \text{si } a \equiv 0 \pmod{p},\\
1 & \text{si } a \not\equiv 0\pmod{p} \text{ y es residuo cuadrático: } \;a\equiv x^2\pmod{p},\\
-1 & \text{si } a \not\equiv 0\pmod{p} \text{ y no es residuo cuadrático.}
\end{array}
\right.
$$

#### Símbolo de Jacobi
El símbolo de Jacobi se define como:

$$
\left(\frac{a}{n}\right) = \left(\frac{a}{p_1}\right)^{\alpha_1}\left(\frac{a}{p_2}\right)^{\alpha_2}\cdots \left(\frac{a}{p_k}\right)^{\alpha_k}
$$

Siendo $n=p_1^{\alpha_1}p_2^{\alpha_2}\cdots p_k^{\alpha_k}$ la factorización de $n$.

#### Ley de reciprocidad cuadrática y propiedades
Tanto el símbolo de Jacobi como el símbolo de Legendre cumplen que:

$$
\left(\frac{m}{n}\right)\left(\frac{n}{m}\right) = (-1)^{\tfrac{m-1}{2}\cdot\tfrac{n-1}{2}}
$$

Usaremos además las siguientes propiedades para su cálculo:

  - $\left(\frac{m}{n}\right) = \left(\frac{m+kn}{n}\right)$, para reducir el "numerador".
  - $\left(\frac{2}{n}\right) = (-1)^{\frac{n^2-1}{8}}$
  - $\left(\frac{1}{n}\right) = 1$, caso base de recursión.

In [1]:
# Utiliza funciones de la librería sympy
from sympy import gcd
from sympy.ntheory.factor_ import factorint

def jacobiSym(m,n):
    
    """Calcula el símbolo de Jacobi de dos enteros. El segundo de ellos
    no debe ser par."""
    
    # Trabaja en módulo n
    m = m % n
    
    # Caso base, no son coprimos
    if gcd(m,n) != 1:
        return 0
    
    # Caso base, 1 es un residuo cuadrático 
    if m == 1:
        return 1
    
    # Caso recursivo, m divisible entre 2
    if m % 2 == 0:
        return pow(-1, (n**2-1)/8) * jacobiSym(m/2, n)
    
    # Caso recursivo, ley de reciprocidad cuadrática
    return pow(-1, ((m-1)*(n-1))/4) * jacobiSym(n,m)

In [15]:
jacobiSym(30, 100000001)

-1

## Pseudoprimos de Euler

Un número es pseudoprimo de Euler respecto de la base $b$, con $mcd(n,b) = 1$ coprimos si cumple que:

In [8]:
def eulerPseudoprime(n,b):
    
    """Comprueba si n es un pseudoprimo de Euler en la base."""
    
    # Eliminamos el caso trivial 2
    if (n%2 == 0): return 2,False
    
    # Comprueba que sean coprimos
    d = gcd(n,b)
    if (d != 1): return d,False
    
    # Comprueba pseudoprimalidad con el símbolo de Jacobi
    return b, pow(b,(n-1)/2,n) == jacobiSym(n,b)